DATA 643 Project 1:

Angus, Pevan, Nathan

We built a simple recommender system for American TV sit-coms. This system will take user input for a sit com and predict the user's preference for possible unwatched sitcoms. This is similar in approach to systems used by Netflix, Hulu and Amazon Prime.

We used Facebook to collect data on 5 popular American TV sitcoms: The Big Bang Theory,	Brooklyn 99, It's Always Sunny in Philadelphia,	The Goldbergs, and The Unbreakable Kimmy Schmidt. Facebook friends were asked to rate these shows from 1 for 'Awful' to 5 for 'Outstanding' and to leave blank un-watched shows using a Google Docs Quiz. The data from the Google We will in turn use these data to generate a basic recommendation system for a test set of withheld data and unwatched shows. 

• Load your data into (for example) an R or pandas dataframe, a Python dictionary or list of lists, (or another data structure of your choosing). From there, create a user-item matrix.
• If you choose to work with a large dataset, you’re encouraged to also create a small, relatively dense “user-item” matrix as a subset so that you can hand-verify your calculations.
• Break your ratings into separate training and test datasets.
• Using your training data, calculate the raw average (mean) rating for every user-item combination.
• Calculate the RMSE for raw average for both your training data and your test data.
• Using your training data, calculate the bias for each user and each item.
• From the raw average, and the appropriate user and item biases, calculate the baseline predictors for every user-item combination.
• Calculate the RMSE for the baseline predictors for both your training data and your test data.
• Summarize your results.

In [168]:
#import desired packages here:
import pandas as pd 

In [169]:
tv_df = pd.read_csv('https://raw.githubusercontent.com/ncooper76/MSDA/master/TV_Show_Preferences.csv')
tv_df = tv_df.drop(columns=["Timestamp"])
tv_df = tv_df[sorted(tv_df.columns)]
tv_df

,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,5.0,NaN,1.0,2.0,3.0
1,2.0,3.0,3.0,2.0,1.0
2,NaN,NaN,4.0,NaN,NaN
3,4.0,NaN,1.0,3.0,5.0
4,NaN,2.0,2.0,NaN,3.0
5,3.0,NaN,1.0,3.0,4.0
6,4.0,1.0,3.0,1.0,2.0
7,NaN,NaN,5.0,NaN,5.0
8,NaN,1.0,3.0,1.0,NaN
9,5.0,NaN,3.0,NaN,NaN


In [170]:
tv_df_mean = tv_df.stack().mean(skipna = True) #stack makes the DF one column, mean takes the mean of that colunm.
print(tv_df_mean) # 3.1279... agrees with the average taken in Google Sheets
#below are the means for the rows and the columns of the dataframe
tv_df_col_means = tv_df.mean(axis=0,skipna = True)
print(tv_df_col_means)
tv_df_row_means = tv_df.mean(axis=1,skipna = True)
print(tv_df_row_means)

3.127906976744186
Brooklyn 99                          3.500000
It's Always Sunny in Philadelphia    2.615385
The Big Bang Theory                  3.347826
The Goldbergs                        2.800000
Unbreakable Kimmy Schmidt            3.157895
dtype: float64
0     2.750000
1     2.200000
2     4.000000
3     3.250000
4     2.333333
5     2.750000
6     2.200000
7     5.000000
8     1.666667
9     4.000000
10    4.000000
11    3.500000
12    3.750000
13    5.000000
14    3.600000
15    4.500000
16    2.400000
17    4.400000
18    3.800000
19    5.000000
20    2.000000
21    3.666667
22    3.000000
23    1.600000
dtype: float64


Now we will break into training and test sets and calculate the raw averages:

In [171]:
from sklearn.model_selection import train_test_split
tv_col_df = tv_df.stack()
tv_col_df.head(20)

tv_train, tv_test = train_test_split(tv_col_df, test_size =0.2)
tv_train = tv_train.sort_index()
tv_test = tv_test.sort_index()
#print(tv_train.head())
#print(tv_test.head())
train_raw_ave = tv_train.mean()
test_raw_ave = tv_test.mean()
print("The Raw Average on the Training Set is: " + str(train_raw_ave) + "\n")
print("The Raw Average on the Test Set is: " + str(test_raw_ave) + "\n")

The Raw Average on the Training Set is: 3.0441176470588234

The Raw Average on the Test Set is: 3.4444444444444446



Now we recast the data frames into their original wide format and calculate the row and column averages to calculate the biases:

In [172]:
tv_train_wide = tv_train.unstack().sort_index()
tv_train_col_means = tv_train_wide.mean(axis=0,skipna = True)
print(str(tv_train_col_means))
tv_train_row_means = tv_train_wide.mean(axis=1,skipna = True)
print(str(tv_train_row_means))

Brooklyn 99                          3.090909
It's Always Sunny in Philadelphia    2.500000
The Big Bang Theory                  3.200000
The Goldbergs                        2.818182
Unbreakable Kimmy Schmidt            3.428571
dtype: float64
0     3.000000
1     2.500000
2     4.000000
3     3.000000
4     2.333333
5     2.750000
6     1.750000
7     5.000000
8     1.666667
9     4.000000
10    4.500000
11    3.000000
12    3.333333
13    5.000000
14    4.000000
15    5.000000
16    2.400000
17    4.333333
18    3.750000
19    5.000000
20    2.000000
21    3.666667
22    2.000000
23    1.750000
dtype: float64


In [173]:
tv_test_wide = tv_test.unstack().sort_index()
tv_test_wide
tv_test_col_means = tv_test_wide.mean(axis=0,skipna = True)
print(str(tv_test_col_means))
tv_test_row_means = tv_test_wide.mean(axis=1,skipna = True)
print(str(tv_test_row_means))

Brooklyn 99                          4.400000
It's Always Sunny in Philadelphia    4.000000
The Big Bang Theory                  4.333333
The Goldbergs                        2.750000
Unbreakable Kimmy Schmidt            2.400000
dtype: float64
0     2.5
1     1.0
3     4.0
6     4.0
10    3.0
11    5.0
12    5.0
14    3.0
15    4.0
17    4.5
18    4.0
20    2.0
22    5.0
23    1.0
dtype: float64


Now we can calculate the RMSE for the raw averages:

In [174]:
from sklearn.metrics import mean_squared_error

train_pred = [tv_train.mean()]*len(tv_train)# we need a list of raw averages the same length as the stacked df
train_pred
train_rmse = mean_squared_error(tv_train, train_pred)**0.5
print("RMSE on the Training Set is: " + str(train_rmse) + "\n")
test_pred = [tv_train.mean()]*len(tv_test) # we need a list of raw averages the same length as the stacked df
test_pred
test_rmse = mean_squared_error(tv_test, test_pred)**0.5
print("RMSE on the Test Set is: " + str(test_rmse) + "\n")

RMSE on the Training Set is: 1.4494808759306146

RMSE on the Test Set is: 1.3192833123651928



Now we will use the row (user) and colunm (item) means to calculate bias, and use those biases to make better predictions for every user-item pair.

In [175]:
show_bias = tv_train_col_means - train_raw_ave 
print(show_bias)
user_bias = tv_train_row_means - train_raw_ave 
print(user_bias) 
tv_pred = pd.DataFrame(index=tv_df.index, columns=tv_df.columns.values)
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
            tv_pred.iloc[i,j] = train_raw_ave + user_bias[i] + show_bias[j]
            if tv_pred.iloc[i,j] < 1.0:
                tv_pred.iloc[i,j] = 1.0
            elif tv_pred.iloc[i,j] > 5.0:
                tv_pred.iloc[i,j] = 5.0
tv_pred

Brooklyn 99                          0.046791
It's Always Sunny in Philadelphia   -0.544118
The Big Bang Theory                  0.155882
The Goldbergs                       -0.225936
Unbreakable Kimmy Schmidt            0.384454
dtype: float64
0    -0.044118
1    -0.544118
2     0.955882
3    -0.044118
4    -0.710784
5    -0.294118
6    -1.294118
7     1.955882
8    -1.377451
9     0.955882
10    1.455882
11   -0.044118
12    0.289216
13    1.955882
14    0.955882
15    1.955882
16   -0.644118
17    1.289216
18    0.705882
19    1.955882
20   -1.044118
21    0.622549
22   -1.044118
23   -1.294118
dtype: float64


,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,3.04679,2.45588,3.15588,2.77406,3.38445
1,2.54679,1.95588,2.65588,2.27406,2.88445
2,4.04679,3.45588,4.15588,3.77406,4.38445
3,3.04679,2.45588,3.15588,2.77406,3.38445
4,2.38012,1.78922,2.48922,2.1074,2.71779
5,2.79679,2.20588,2.90588,2.52406,3.13445
6,1.79679,1.20588,1.90588,1.52406,2.13445
7,5,4.45588,5,4.77406,5
8,1.71346,1.12255,1.82255,1.44073,2.05112
9,4.04679,3.45588,4.15588,3.77406,4.38445


In [176]:
import math
tv_pred_train = pd.DataFrame(index=tv_df.index, columns=tv_df.columns.values)
tv_pred_train = tv_pred_train[sorted(tv_pred_train.columns)]
tv_train_wide = tv_train_wide[sorted(tv_train_wide.columns)]
for i in range(0,len(user_bias)):
    for j in range(0,len(show_bias)):
        if math.isnan(tv_train_wide.iloc[i,j]):
            continue
        else:
            tv_pred_train.iloc[i,j] = train_raw_ave + user_bias[i] + show_bias[j]
            if tv_pred_train.iloc[i,j] < 1.0:
                tv_pred_train.iloc[i,j] = 1.0
            elif tv_pred_train.iloc[i,j] > 5.0:
                tv_pred_train.iloc[i,j] = 5.0
tv_pred_train

,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,3.04679,NaN,3.15588,NaN,NaN
1,2.54679,1.95588,2.65588,2.27406,NaN
2,NaN,NaN,4.15588,NaN,NaN
3,NaN,NaN,3.15588,2.77406,3.38445
4,NaN,1.78922,2.48922,NaN,2.71779
5,2.79679,NaN,2.90588,2.52406,3.13445
6,NaN,1.20588,1.90588,1.52406,2.13445
7,NaN,NaN,5,NaN,5
8,NaN,1.12255,1.82255,1.44073,NaN
9,4.04679,NaN,4.15588,NaN,NaN


In [177]:
train_bias_rmse = mean_squared_error(tv_train, tv_pred_train.stack().sort_index())**0.5
print("RMSE on the Training Set is: " + str(train_bias_rmse) + "\n")

RMSE on the Training Set is: 1.0402064392907835



In [178]:
tv_pred_test = pd.DataFrame(index=tv_df.index, columns=tv_df.columns.values)
tv_pred_test = tv_pred_test[sorted(tv_pred_test.columns)]
tv_test_wide = tv_test_wide[sorted(tv_test_wide.columns)]
tv_test_wide = tv_test_wide.reindex(tv_df.index)
for l in range(0,len(user_bias)):
    for k in range(0,len(show_bias)):
        if math.isnan(tv_test_wide.iloc[l,k]):
            continue
        else:
            tv_pred_test.iloc[l,k] = test_raw_ave + user_bias[l] + show_bias[k]
            if tv_pred_test.iloc[l,k] < 1.0:
                tv_pred_test.iloc[l,k] = 1.0
            elif tv_pred_test.iloc[l,k] > 5.0:
                tv_pred_test.iloc[l,k] = 5.0
tv_pred_test

,Brooklyn 99,It's Always Sunny in Philadelphia,The Big Bang Theory,The Goldbergs,Unbreakable Kimmy Schmidt
0,NaN,NaN,NaN,3.17439,3.78478
1,NaN,NaN,NaN,NaN,3.28478
2,NaN,NaN,NaN,NaN,NaN
3,3.44712,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,2.19712,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [179]:
test_bias_rmse = mean_squared_error(tv_test, tv_pred_test.stack().sort_index())**0.5
print("RMSE on the Test Set is: " + str(test_bias_rmse) + "\n")

RMSE on the Test Set is: 1.3698792868204648

